In this model we are using convLSTM to predict the next frame of a video based on past video sequences. We use convLSTM, which is a LSTM layer but the input transformation and gate operations are replaced by convolutional operations, that allow to capture spatio-temporal dependencies. The input video will be the downsampled version of the original video, with size 8x8 pixels. 

In [7]:
import os
import h5py
import numpy as np

## Pre process the data

In [3]:
dir=os.listdir('/home/catia/Desktop/zebrafish/data')
dir.sort()

In [8]:
minimum=1000
maximum=0
selected_slice=23
j=0
for i in range(selected_slice,len(dir),100):
    im=h5py.File('data/'+dir[i],'r+')
    img=list(im['d1']) 
    #print (dir[i])
    new_min=np.min(img)
    new_max=np.max(img)
    if new_min < minimum:
        minimum=new_min
        #print (minimum)
    if new_max > maximum:
        maximum=new_max
        #print (maximum)
    j=j+1
    
    
num_img=j

In [86]:
data=np.zeros((num_img,400,400))
j=0

# Data starts by having values between minimum and maximum
for i in range (selected_slice,len(dir),100):
    im=h5py.File('data/'+dir[i],'r+')
    img=list(im['d1']) 
    img=img-minimum # So the data is between 0 and maximum
    img=img/(0.5*(maximum-minimum)) # So the data is between 0 and 2
    img=img/2 # So the data is between -1 and 1
    data[j,:,:]=img # Store the image into the variable data
    j=j+1
    

In [87]:
down_data=np.zeros((num_img,8,8))
for i in range(0,len(down_data)):
    img=data[i,:,:]
    dimg=np.zeros((8,8))
    for j in range (0,8):
        for k in range (0,8):
            dimg[j,k]=np.mean(img[j*50:j*50+50,k*50:k*50+50])
    down_data[i,:,:]=dimg

In [88]:
down_data.shape

(8839, 8, 8)

The network can't receive the entire training data set at once, so we have to reshape the data to (sequence,timesteps,row,col,1), `sequence` is the number of sequence examples, `timesteps` is how far into the past we are looking in each example, and `row`,`col`and `1` is the shape of the images since we only have one channel (gray scale images).

In [89]:
x_train_data=down_data[0:7000,:,:]

In [ ]:
x_train_data.shape

(7000, 8, 8)

In [ ]:
past=20 #how far into the past we are looking
input_data=np.zeros((len(x_train_data)-past,past,x_train_data.shape[1],x_train_data.shape[2],1))

for i in range(past,len(x_train_data)):
    input_data[i-past,:,:,:,0]=x_train_data[i-past:i,:,:]
    

In [ ]:
x_train_data[20-20:20,:,:].shape

(20, 8, 8)

In [ ]:
input_data.shape

(6980, 20, 8, 8, 1)

In [ ]:
input_data[4,:,:,:,0].shape

(20, 8, 8)

In [ ]:
y_train_data=down_data[past+1:7001]

In [ ]:
input_data.shape

(6980, 20, 8, 8, 1)

In [ ]:
y_train_data=y_train_data.reshape((6980, 1, 8, 8, 1))

In [ ]:
y_train_data.shape

(6980, 1, 8, 8, 1)

## Model

In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import numpy as np
import pylab as plt

In [ ]:
seq = Sequential()
seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   input_shape=(None, 8, 8, 1),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
               activation='sigmoid',
               padding='same', data_format='channels_last'))
seq.compile(loss='binary_crossentropy', optimizer='adadelta',metrics=['accuracy'])


In [ ]:
seq.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_29 (ConvLSTM2D) (None, None, 8, 8, 40)    59200     
_________________________________________________________________
batch_normalization_29 (Batc (None, None, 8, 8, 40)    160       
_________________________________________________________________
conv_lst_m2d_30 (ConvLSTM2D) (None, None, 8, 8, 40)    115360    
_________________________________________________________________
batch_normalization_30 (Batc (None, None, 8, 8, 40)    160       
_________________________________________________________________
conv_lst_m2d_31 (ConvLSTM2D) (None, None, 8, 8, 40)    115360    
_________________________________________________________________
batch_normalization_31 (Batc (None, None, 8, 8, 40)    160       
_________________________________________________________________
conv_lst_m2d_32 (ConvLSTM2D) (None, None, 8, 8, 40)    115360    
__________

In [ ]:
seq.fit(input_data, y_train_data, batch_size=10,
        epochs=30, validation_split=0.05)

Train on 6631 samples, validate on 349 samples
Epoch 1/30
6631/6631 [==============================] - 1037s 156ms/step - loss: 0.3354 - acc: 0.0000e+00 - val_loss: 0.3293 - val_acc: 0.0000e+00
Epoch 2/30
2500/6631 [==========>...................] - ETA: 10:31 - loss: 0.3319 - acc: 0.0000e+00

In [ ]:
y_train_data.shape